# Sentiment Analysis

In [ ]:
import csv

import pandas as pd

import utils

from datasets import load_dataset, Dataset

Link to dataset: http://huggingface.co/datasets/stanfordnlp/sst2/viewer/default/train?row=2&views%5B%5D=train

In [13]:
sentiment_train_data_raw = load_dataset("stanfordnlp/sst2", trust_remote_code=True, split="train")

In [23]:
sentiment_train_data_file = r"C:\Users\jayde\OneDrive\School\Text Mining for AI\final_project_tm\data\train_data\sentiment-train.tsv"

In [26]:
def sentiment_data_to_file(raw_data: Dataset = sentiment_train_data_raw, file_name: str = sentiment_train_data_file):
    with open(file_name, "w", newline="", encoding="utf-8") as f:
        writer: csv.writer = csv.writer(f, delimiter="\t")
        writer.writerow(["sentence_id", "sentence", "sentiment"])
        
        for idx, elem in enumerate(raw_data):
            sentence = elem["sentence"]
            label = "positive" if elem["label"] == 1 else "negative"
            
            writer.writerow([idx, sentence, label])
        
    f.close()        

In [ ]:
utils.sentiment_data_to_file(raw_data=sentiment_train_data_raw, file_name=sentiment_train_data_file)

Call the following to convert the data to a CSV file:

```python
utils.sentiment_data_to_file(raw_data, file_name)
```

In [28]:
df: pd.DataFrame = pd.read_csv(sentiment_train_data_file, sep="\t")
df.head()

,sentence_id,sentence,sentiment
0,0,hide new secretions from the parental units,negative
1,1,"contains no wit , only labored gags",negative
2,2,that loves its characters and communicates som...,positive
3,3,remains utterly satisfied to remain the same t...,negative
4,4,on the worst revenge-of-the-nerds clichés the ...,negative


## Train the Model

In [29]:
# Next part ...